<a href="https://colab.research.google.com/github/yougoal/leejunghoon/blob/master/cart20152R.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#참조: https://hunkim.github.io/ml/
#setup
import numpy as np
import tensorflow as tf
import random
from collections import deque

import gym
env = gym.make('CartPole-v0')

input_size = env.observation_space.shape[0]
output_size = env.action_space.n
dis = 0.9
REPLAY_MEMORY = 500
class DQN:
    def __init__(self, session, input_size, output_size, name="main"):
        self.session = session
        self.input_size = input_size
        self.output_size = output_size
        self.net_name = name
        self._build_network()
    def _build_network(self, h_size=10, l_rate=1e-1):
        with tf.variable_scope(self.net_name):
            self._X = tf.placeholder(tf.float32, [None, self.input_size], name="input_x")
            W1 = tf.get_variable("W1", shape = [self.input_size, h_size], initializer = tf.contrib.layers.xavier_initializer())
            layer1 = tf.nn.tanh(tf.matmul(self._X, W1))
            W2 = tf.get_variable("W2", shape=[h_size, self.output_size], initializer = tf.contrib.layers.xavier_initializer())
            self._Qpred = tf.matmul(layer1, W2)
        self._Y = tf.placeholder(shape = [None, self.output_size], dtype = tf.float32)
        self._loss = tf.reduce_mean(tf.square(self._Y - self._Qpred))
        self._train = tf.train.AdamOptimizer(learning_rate=l_rate).minimize(self._loss)
    def predict(self, state):
        x = np.reshape(state, [1, self.input_size])
        return self.session.run(self._Qpred, feed_dict={self._X: x})
    def update(self, x_stack, y_stack):
        return self.session.run([self._loss, self._train], feed_dict={self._X: x_stack, self._Y: y_stack})
def replay_train(mainDQN, targetDQN, train_batch):
    x_stack = np.empty(0).reshape(0, input_size)
    y_stack = np.empty(0).reshape(0, output_size)
    for state, action, reward, next_state, done in train_batch:
        Q = mainDQN.predict(state)
        if done:
            Q[0, action] = reward
        else:
            Q[0, action] = reward + dis * np.max(targetDQN.predict(next_state))
        y_stack = np.vstack([y_stack, Q])
        x_stack = np.vstack([x_stack, state])
    return mainDQN.update(x_stack, y_stack)
def bot_play(mainDQN):
    s = env.reset()
    reward_sum = 0
    while True:
        #env.render()
        a = np.argmax(mainDQN.predict(s))
        s, reward, done, _ = env.step(a)
        reward_sum += reward
        if done:
            print("Total score: {}".format(reward_sum))
            break
def get_copy_var_ops(*, dest_scope_name="target", src_scope_name="main"):
    op_holder = []
    src_vals = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope = src_scope_name)
    dest_vals = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope = dest_scope_name)

    for src_vals, dest_vals in zip(src_vals, dest_vals):
        op_holder.append(dest_vals.assign(src_vals.value()))
    return op_holder
print("이정훈4")

max_episodes = 10
replay_buffer = deque()
with tf.Session() as sess:
    mainDQN = DQN(sess, input_size, output_size, name="main")
    targetDQN = DQN(sess, input_size, output_size, name="target")
    tf.global_variables_initializer().run()
    copy_ops = get_copy_var_ops(dest_scope_name="target", src_scope_name="main")
    sess.run(copy_ops)
    for episode in range(max_episodes):
        e = 1. / ((episode / 10) + 1)
        done = False
        step_count = 0

        state = env.reset()
        print("이정훈5")
        while not done:

            if np.random.rand(1) < e :
                action = env.action_space.sample()
            else:
                action = np.argmax(mainDQN.predict(state))

            next_state, reward, done, _ = env.step(action)

            if done:
                reward = -100

            replay_buffer.append((state, action, reward, next_state, done))

            if len(replay_buffer) > REPLAY_MEMORY:
                replay_buffer.popleft()

            state = next_state
            step_count += 1
            if step_count > 10000:
                break

        print("episode: {}  step: {}".format(episode, step_count))

        if step_count > 10000:
            pass

        if episode % 10 == 1:
            for _ in range(50):
                minibatch = random.sample(replay_buffer, 10)
                loss, _ = replay_train(mainDQN, targetDQN, minibatch)
            print("Loss: ", loss)
            sess.run(copy_ops)

    bot_play(mainDQN)



이정훈4
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



/usr/local/lib/python3.6/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


이정훈5
episode: 0  step: 14
이정훈5
episode: 1  step: 18
Loss:  12.13175
이정훈5
episode: 2  step: 20
이정훈5
episode: 3  step: 100
이정훈5
episode: 4  step: 36
이정훈5
episode: 5  step: 24
이정훈5
episode: 6  step: 56
이정훈5
episode: 7  step: 70
이정훈5
episode: 8  step: 61
이정훈5
episode: 9  step: 109
Total score: 177.0
